In [107]:
import importlib
import toolbox
importlib.reload(toolbox)
#from ml_eda import edaDF

<module 'toolbox' from 'c:\\Users\\msieb\\OneDrive\\Documents\\1 NAIT\\1 - DATA3950 MACHINE LEARNING\\Repository\\3950-assignment-2-mlsiebold\\toolbox.py'>

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import SGDRegressor

from sklearn.pipeline import Pipeline

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV

from ml_utils import edaDF
import toolbox as tb

In [3]:
# Download and load the data
import keras
import os

f_path_1 = "data/Euro_Salary.csv"
url_1 = "https://github.com/AkeemSemper/ml_data/raw/main/Euro_Salary.csv"
if not os.path.exists(f_path_1):
    file_1 = keras.utils.get_file(f_path_1, url_1)
    
df = pd.read_csv(f_path_1)

# Assignment 2 - Regression
### Predict the TOTAL COMPENSATION for this year. 

The data file provided is a salary survey for tech workers in Europe. We want to predict the total amount of compensation they bring in each year, based off of the details of their work. 

Some notes that will be important:
<ul>
    <li>The total compensation will need to be constructed, there is a column for salary, "Yearly brutto salary (without bonus and stocks) in EUR", as well as a column for bonus compensation, "Yearly bonus + stocks in EUR". 
    <li>Some categorical variables will need some work, and there isn't generally an exact answer. The main concern is things with categories that have a bunch of values with a very small count. For example, if there is only 1 person in City X, then that value likely needs to be addressed. We don't want it encoded into a new column of one 1 and thousands of 0s. 
    <li>There is an article exploring some of the data here: https://www.asdcode.de/2021/01/it-salary-survey-december-2020.html
    <li>Imputation and a bit of data manipulation will be required. 
    <li>Use any regression method you'd like. Some ones are closely related to what we've done, you may want to look at them, e.g. ExtraTreesRegressor. 
    <li>Initial accurracy, and potentially final accuracy, may not be great. When I made a plain model will little optimization the errors were large and the R2 was low. There is lots of room for optimization. 
    <li>Research challenge - try some work on the target, look into TransformedTargetRegressor and see if that helps. Recall in stats when we had skewed distributions... Maybe it helps, maybe it doesn't. 
    <li>EDA and data prep are up to you - you'll probably need to do a little exploring to figure out what cleanup is needed. When I did it, I did things kind of iteratively when I did it. For example, look at the value counts, figure out how to treat the different categories, clean something up, look at the results, potentially repeat if needed. After you figure out what needs to be done, you may be able to take some of those steps and incorporate them into a pipeline to be cleaner....
    <li><b>CRITICAL - Please make sure you publish it after having run it, all the output should be showing.</b>
</ul>

### Details and Deliverables

You'll need to build code to produce the predictions. In particular, there's a few things that'll be marked:
<ul>
    <li>Please add a "presentation version" at the bottom, where you show what you did, and the results. Basically, you start with the original data, you do some work to figure out what's needed, you try a few models and select the best. At the bottom, put what <i>you actually settled on</i>, i.e. after all the figuring and exploring, here's the code that goes from raw data to final results, and here's what the results were. I should be able to read this part and understand what you did clearly:
    <ul>
        <li> Please make a pipeline that does the prep work - you may need some exploration or several trials before settling on what exactly to use, that's normal. Once you've settled, build that into a pipeline so it's clear and repeatable.
        <li> What you settled on for data cleaning, along with what prompted it. 
        <li> Feature Selection - Please identify what you did for feature selection. No need for a long explaination, something along the lines of "I did X, and the result was that 4 features were removed". Try at least 2 things. 
        <li> Model selection - between selecting a model style and tuning it with hyperparameters, what did you test and what won?
        <li> Overall, how good was your model and what things may make sense to try to do even better? 
        <li> If you could use titles/bullet points I'd really appreciate it. 
    </ul>
    <li>Grade Breakdown:
    <ul>
        <li> Code is readable, there are comments: 20%
        <li> Explaination as defined above: 60% (20% each point)
        <li> Accuracy: 20% As compared to everyone else. This will be generously graded, I won't be surprised if overall accuracy is low for most people. 
    </ul>
</ul>

<b>The biggest challenge here is translating the data into something useful and clean. This will probably require a bit of exploration, examining the data, thinking about what it means, trying something, then making a model to see what the results are. In particular, think about what value some of the less clean bits of data my hold - binning/grouping, numerical transformations, outlier removal, etc... are all likely to be useful somewhere. You almost certainly need to look at it column by column and make a decision, I'll apologize up front, it isn't the most fun process in the world. There is not one specific correct answer.</b>

### Basic Cleaning

In [ ]:
#  Remove whitespaces and weird characters

import unicodedata

df = df.apply(lambda col: col.map(tb.clean_string))     # Clean entire df
df.columns = df.columns.map(tb.clean_string)            # Clean column headers

In [ ]:
# Replace null-like values with NaN

tb.clean_null_like(df, columns=None)

### Data Dictionary

In [5]:
# Create data dictionary

data_dict = pd.DataFrame({
    'Dtype': df.dtypes,
    'Semantic Type': None,
    'Desc': None,
    '# of Nulls': df.isna().sum(),
    '# of Unique Values': df.nunique()
    })                                      #.reset_index(names='Column')

data_dict.index.name = 'Column'             # Give the index a name

In [6]:
# Create description dictionary to add manually input column descriptions to data dictionary

# Create list of columns
cols = df.columns.to_list()

# Build description dictionary scaffold
desc_dict = {col: '' for col in cols}
#desc_dict

# Build description dictionary scaffold
semantic_dict = {col: '' for col in cols}
#semantic_dict

In [8]:
# Manually update description dictionary

desc_dict = {
 "Timestamp": "Date and time survey response was submitted (acts as survey submission identifier)",
 "Age": "Respondent's age in years",
 "Gender": "Respondent's self-report gender",
 "City": "City in which the respondent works (primarily German cities, with a few internationaly cities)",
 "Position": "Current job title or primary role",
 "Total years of experience": "Total number of years the respondent has worked in their profession",
 "Years of experience in Germany": "Number of years the respondent has worked specifically in Germany",
 "Seniority level": "Self‑reported seniority",
 "Your main technology / programming language": "Primary programming language or technology used in the respondent’s job",
 "Other technologies/programming languages you use often": "Additional languages, frameworks, or tools the respondent frequently uses",
 "Yearly brutto salary (without bonus and stocks) in EUR": "Annual gross base salary in euros, excluding bonuses and stock compensation",
 "Yearly bonus + stocks in EUR": "Annual bonus and/or stock compensation in euros",
 "Number of vacation days": "Number of paid vacation days per year",
 "Employment status": "Type of employment",
 "Contract duration": "Type of employment contract",
 "Main language at work": "Primary language used in the workplace",
 "Company size": "Approximate number of employees in the respondent’s company",
 "Company type": "Type of organization respondent is employed for"}

In [11]:
# Manually update semantic type dictionary

semantic_dict = {
 'Timestamp': 'categorical nominal',
 'Age': 'numeric discrete',
 'Gender': 'categorical nominal',
 'City': 'categorical nominal',
 'Position': 'categorical nominal',
 'Total years of experience': 'numeric continuous',
 'Years of experience in Germany': 'numeric continuous',
 'Seniority level': 'categorical ordinal',
 'Your main technology / programming language': 'categorical nominal',
 'Other technologies/programming languages you use often': 'categorical nominal',
 'Yearly brutto salary (without bonus and stocks) in EUR': 'numeric continuous',
 'Yearly bonus + stocks in EUR': 'numeric continuous',
 'Number of vacation days': 'numeric discrete',
 'Employment status': 'categorical nominal',
 'Contract duration': 'categorical nominal',
 'Main language at work': 'categorical nominal',
 'Company size': 'categorical ordinal',
 'Company type': 'categorical nominal'}

In [12]:
# Map descriptions and data type to data dictionary 

#data_dict['Desc'] = data_dict['Column'].map(desc_dict)
#data_dict['Semantic Type'] = data_dict['Column'].map(semantic_dict)

data_dict['Desc'] = data_dict.index.map(desc_dict)
data_dict['Semantic Type'] = data_dict.index.map(semantic_dict)

In [13]:
# Check my handywork

data_dict

,Dtype,Semantic Type,Desc,# of Nulls,# of Unique Values
Column,,,,,
Timestamp,object,categorical nominal,Date and time survey response was submitted (a...,0,1248
Age,float64,numeric discrete,Respondent's age in years,27,40
Gender,object,categorical nominal,Respondent's self-report gender,10,3
City,object,categorical nominal,City in which the respondent works (primarily ...,0,107
Position,object,categorical nominal,Current job title or primary role,6,141
Total years of experience,object,numeric continuous,Total number of years the respondent has worke...,16,48
Years of experience in Germany,object,numeric continuous,Number of years the respondent has worked spec...,32,52
Seniority level,object,categorical ordinal,Self‑reported seniority,12,23
Your main technology / programming language,object,categorical nominal,Primary programming language or technology use...,127,243


In [14]:
# Save data dictionary to markdown file

data_dict.reset_index().to_csv('data_dictionary.tsv', sep='\t', index=False, encoding='utf-8-sig')

### Convert Dtypes

In [93]:
# Change dtypes based on semantic type

df = tb.apply_semantic_dtypes(df, data_dict, tb.semantic_to_dtype)

In [94]:
df.dtypes

Column
Timestamp                                                 category
Age                                                        float64
Gender                                                    category
City                                                      category
Position                                                  category
Total years of experience                                  float64
Years of experience in Germany                             float64
Seniority level                                           category
Your main technology / programming language               category
Other technologies/programming languages you use often    category
Yearly brutto salary (without bonus and stocks) in EUR     float64
Yearly bonus + stocks in EUR                               float64
Number of vacation days                                    float64
Employment status                                         category
Contract duration                                      

### Basic EDA

In [108]:
# Check for duplicates

df.duplicated().value_counts()

False    1253
Name: count, dtype: int64

In [109]:
# Basic stats

df.describe(include="all").T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Column,,,,,,,,,,,
Timestamp,1253,1248,24/11/2020 15:07:35,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Age,1226.0,NaN,NaN,NaN,32.509788,5.663804,20.0,29.0,32.0,35.0,69.0
Gender,1243,3,male,1049,NaN,NaN,NaN,NaN,NaN,NaN,NaN
City,1253,106,berlin,681,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Position,1247,133,software engineer,387,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Total years of experience,1230.0,NaN,NaN,NaN,9.07813,11.901317,0.0,5.0,8.0,12.0,383.0
Years of experience in Germany,1195.0,NaN,NaN,NaN,3.754812,3.656636,0.0,1.0,3.0,5.0,30.0
Seniority level,1241,21,senior,565,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Your main technology / programming language,1125,206,python,212,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [110]:
# Number of unique values per feature

print('\033[1m# of unique values:\033[0m')
print(df.nunique())

# of unique values:
Column
Timestamp                                                 1248
Age                                                         40
Gender                                                       3
City                                                       106
Position                                                   133
Total years of experience                                   42
Years of experience in Germany                              36
Seniority level                                             21
Your main technology / programming language                206
Other technologies/programming languages you use often     561
Yearly brutto salary (without bonus and stocks) in EUR     201
Yearly bonus + stocks in EUR                               162
Number of vacation days                                     36
Employment status                                           10
Contract duration                                            3
Main language at work       

### Column Specific EDA

In [114]:
# Set column lists

num_cols = tb.detect_num_cols(df)
cat_cols = tb.detect_cat_cols(df)

### **'Timestamp'**
- **Inconsistencies:**
    - dtype = category
- **Actions taken:**
    - convert dtype to DateTime

In [129]:
# Basic EDA

column = 'Timestamp'
tb.eda_column_summary(df, column)

=== COLUMN: Timestamp ===

DTYPE:
category

NON-NULL VALUES:
1,253

NULL VALUES:
0

UNIQUE VALUES:
1,248

BASIC STATS:
count                    1253
unique                   1248
top       24/11/2020 15:07:35
freq                        2
Name: Timestamp, dtype: object

TOP VALUES:
Timestamp
24/11/2020 15:07:35    2
24/11/2020 13:55:19    2
24/11/2020 14:07:23    2
25/11/2020 08:47:37    2
25/11/2020 18:28:01    2
Name: count, dtype: int64

BOTTOM VALUES:
Timestamp
30/11/2020 19:28:15    1
30/11/2020 20:09:04    1
30/11/2020 20:14:04    1
30/11/2020 20:39:33    1
30/11/2020 20:56:33    1
Name: count, dtype: int64


In [130]:
df[df[column].duplicated(keep=False)].sort_values(by=column)

Column,Timestamp,Age,Gender,City,Position,Total years of experience,Years of experience in Germany,Seniority level,Your main technology / programming language,Other technologies/programming languages you use often,Yearly brutto salary (without bonus and stocks) in EUR,Yearly bonus + stocks in EUR,Number of vacation days,Employment status,Contract duration,Main language at work,Company size,Company type
234,24/11/2020 13:55:19,25.0,male,munich,ml engineer,1.0,1.0,middle,python,"python, c/c++, swift, matlab",65000.0,2000.0,30.0,full-time employee,unlimited contract,english,1000+,product
235,24/11/2020 13:55:19,32.0,male,berlin,engineering manager,9.0,5.0,lead,<NA>,"php, javascript / typescript, .net, sql, googl...",95000.0,140000.0,365.0,full-time employee,unlimited contract,english,1000+,product
246,24/11/2020 14:07:23,34.0,male,berlin,software engineer,14.0,5.0,senior,c#,"javascript / typescript, .net, sql, aws, docker",72000.0,NaN,26.0,full-time employee,unlimited contract,english,101-1000,product
247,24/11/2020 14:07:23,37.0,male,berlin,team lead,12.0,10.0,head,python,"python, php, javascript / typescript, sql, docker",90000.0,NaN,30.0,full-time employee,unlimited contract,english,11-50,consulting / agency
280,24/11/2020 15:07:35,33.0,male,frankfurt,software engineer,10.0,1.0,senior,.net,"javascript / typescript, sql, azure",26400.0,NaN,NaN,self-employed (freelancer),unlimited contract,english,up to 10,startup
281,24/11/2020 15:07:35,28.0,male,berlin,frontend developer,8.0,5.0,senior,javascript,<NA>,68000.0,NaN,NaN,full-time employee,<NA>,<NA>,<NA>,<NA>
490,25/11/2020 08:47:37,29.0,male,munich,software engineer,8.0,6.0,middle,php,"javascript / typescript, sql",75000.0,NaN,30.0,full-time employee,unlimited contract,english,1000+,product
491,25/11/2020 08:47:37,37.0,male,malta,software architect,18.0,NaN,head,go,"google cloud, kubernetes, docker, shell",156000.0,0.0,0.0,self-employed (freelancer),unlimited contract,english,11-50,startup
665,25/11/2020 18:28:01,32.0,male,berlin,qa engineer,10.0,3.0,senior,js,"java / scala, google cloud, docker",81000.0,45000.0,30.0,full-time employee,unlimited contract,english,1000+,product
666,25/11/2020 18:28:01,32.0,male,cologne,devops,7.0,2.0,middle,k8s,"python, google cloud, kubernetes, docker",60000.0,NaN,30.0,full-time employee,unlimited contract,english,101-1000,product


In [133]:
df[column] = pd.to_datetime(df[column], errors='coerce')

C:\Users\msieb\AppData\Local\Temp\ipykernel_28856\3686196875.py:1: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df[column] = pd.to_datetime(df[column], errors='coerce')


In [138]:
df[column].sort_values(ascending=False)

1252   2021-01-19 12:01:11
1251   2021-01-19 10:17:58
1250   2021-01-18 23:20:35
1249   2021-01-18 17:46:02
1248   2021-01-18 15:05:50
               ...        
4      2020-11-24 11:15:46
3      2020-11-24 11:15:24
2      2020-11-24 11:14:21
1      2020-11-24 11:14:16
0      2020-11-24 11:14:15
Name: Timestamp, Length: 1253, dtype: datetime64[ns]

In [ ]:
#tb.plot(df, columns=column)

- Duplicate Timestamp's look fine. No duplicate records
- Can't see any use for this column so drop it


### **'Age'**
- **Inconsistencies:**
    - dtype is float64, should be integer
- **Actions taken:**
    - Change dtype to integer

In [112]:
# Basic EDA

tb.eda_column_summary(df, 'Age')

=== COLUMN: Age ===

DTYPE:
float64

NON-NULL VALUES:
1,226

NULL VALUES:
27

UNIQUE VALUES:
40

BASIC STATS:
count    1226.000000
mean       32.509788
std         5.663804
min        20.000000
25%        29.000000
50%        32.000000
75%        35.000000
max        69.000000
Name: Age, dtype: float64

TOP VALUES:
Age
30.0    110
32.0     94
33.0     94
28.0     87
31.0     87
Name: count, dtype: int64

BOTTOM VALUES:
Age
50.0    1
21.0    1
20.0    1
69.0    1
52.0    1
Name: count, dtype: int64


### **'Gender'**
- **Inconsistencies:**
    - None
- **Actions taken:**
    - None

In [35]:
# Basic EDA

tb.eda_column_summary(df, 'Gender')

=== COLUMN: Gender ===

DTYPE:
string

NON-NULL VALUES:
1,243

NULL VALUES:
10

BASIC STATS:
count     1243
unique       3
top       male
freq      1049
Name: Gender, dtype: object

TOP VALUES:
Gender
male       1049
female      192
diverse       2
Name: count, dtype: Int64

BOTTOM VALUES:
Gender
male       1049
female      192
diverse       2
Name: count, dtype: Int64


In [ ]:
np.sort(df['City'].unique()).tolist()

['Aachen',
 'Amsterdam',
 'Ansbach',
 'Barcelona',
 'Basel',
 'Berlin',
 'Bielefeld',
 'Bodensee',
 'Boeblingen',
 'Bonn',
 'Braunschweig ',
 'Brunswick',
 'Brussels',
 'Brussels ',
 'Bucharest',
 'Bölingen',
 'Cambridge',
 'City in Russia',
 'Cologne',
 'Copenhagen',
 'Cracovia',
 'Cracow',
 'Cupertino',
 'Darmstadt',
 'Den Haag',
 'Dortmund',
 'Dresden',
 'Dublin',
 'Dublin ',
 'Duesseldorf',
 'Dusseldorf',
 'Dusseldurf',
 'Düsseldorf',
 'Düsseldorf ',
 'Eindhoven',
 'Erlangen',
 'Fr',
 'France',
 'Frankfurt',
 'Friedrichshafen',
 'Hamburg',
 'Hannover',
 'Heidelberg',
 'Heidelberg ',
 'Heilbronn',
 'Helsinki',
 'Hildesheim',
 'Hildesheim ',
 'Ingolstadt',
 'Ingolstadt ',
 'Innsbruck',
 'Istanbul',
 'Jena',
 'Karlsruhe',
 'Karlsruhe ',
 'Kempten',
 'Kiev',
 'Koblenz',
 'Konstanz',
 'Konstanz area',
 'Krakow',
 'Kyiv',
 'Köln',
 'Leipzig',
 'Lisbon',
 'London',
 'Luttich',
 'Lübeck',
 'Madrid',
 'Malta',
 'Mannheim',
 'Mannheim ',
 'Marseille',
 'Milan',
 'Milano',
 'Minsk',
 'Moldova

### **'City'**
- **Inconsistencies:**
    - Countries rather than cities
    - Special characters ex. "()"
    - Trailing spaces
    - Invalid answers and extra text ("Prefer not to say", "area", "City in"
    - Abreviations
    - Spelling errors
    - Multiple spellings
    - Accents    
- **Actions taken:**
    - Removed trailing spaces
    - 

In [36]:
# Basic EDA

tb.eda_column_summary(df, 'City')

=== COLUMN: City ===

DTYPE:
string

NON-NULL VALUES:
1,253

NULL VALUES:
0

BASIC STATS:
count       1253
unique       106
top       berlin
freq         681
Name: City, dtype: object

TOP VALUES:
City
berlin       681
munich       236
frankfurt     44
hamburg       40
stuttgart     33
Name: count, dtype: Int64

BOTTOM VALUES:
City
duesseldorf    1
copenhagen     1
sevilla        1
krakow         1
saarbrucken    1
Name: count, dtype: Int64


In [ ]:
df['City'].unique()

array(['Munich', 'Berlin', 'Hamburg', 'Wolfsburg', 'Stuttgart',
       'Schleswig-Holstein', 'London', 'Konstanz area', 'Frankfurt',
       'Cologne', 'Kempten', 'Münster', 'Erlangen', 'Vienna', 'Moldova',
       'Rosenheim', 'Mannheim ', 'Boeblingen', 'Düsseldorf', 'Ingolstadt',
       'Nürnberg', 'Ansbach', 'Leipzig', 'Mannheim', 'Tuttlingen', 'Bonn',
       'Moscow', 'Koblenz', 'Warsaw', 'Heidelberg', 'Karlsruhe', 'Köln',
       'Aachen', 'Karlsruhe ', 'Samara', 'Riga, Latvia', 'Dusseldorf',
       'Zurich', 'Helsinki', 'Würzburg', 'Kiev', 'Den Haag', 'Amsterdam',
       'Cracovia', 'Tallinn', 'Prague', 'Utrecht', 'Stockholm',
       'Braunschweig ', 'Dresden', 'Kyiv', 'Stuttgart ', 'Malta',
       'Lübeck', 'Nuremberg ', 'Bodensee', 'Milan', 'Salzburg', 'Rome',
       'Wroclaw', 'Cupertino', 'Paris', 'Dublin ', 'Paderborn',
       'Konstanz', 'Ulm', 'Düsseldorf ', 'Barcelona', 'Bölingen',
       'Tampere (Finland)', 'Hannover', 'Bucharest', 'Siegen', 'Minsk',
       'Nuremberg', 'M

In [ ]:
# Remove accents

from unidecode import unicodedata

print(f"# of unique values BEFORE normalizing accents: {len(df['City'].unique())}")

df['City'] = df['City']    .str.encode('ascii', 'ignore') 

print(f"# of unique values AFTER normalizing accents: {len(df['City'].unique())}")

ModuleNotFoundError: No module named 'unidecode'

In [ ]:
df["City"] = df["City"].replace({
    "Dusseldurf": "Düsseldorf",
    "Koln": "Cologne",
    "Cracovia": "Krakow"
})

### 'Position'
- **Definition:**
    - 
- **Inconsistencies:**
    -
- **Actions taken:**
    - 

In [37]:
# Basic EDA

tb.eda_column_summary(df, 'Position')

=== COLUMN: Position ===

DTYPE:
string

NON-NULL VALUES:
1,247

NULL VALUES:
6

BASIC STATS:
count                  1247
unique                  133
top       software engineer
freq                    387
Name: Position, dtype: object

TOP VALUES:
Position
software engineer     387
backend developer     174
data scientist        110
frontend developer     89
qa engineer            71
Name: count, dtype: Int64

BOTTOM VALUES:
Position
software architekt                       1
testmanager                              1
erp consultant                           1
researcher/ consumer insights analyst    1
it operations manager                    1
Name: count, dtype: Int64


###  'Total years of experience'
- **Definition:**
    - 
- **Inconsistencies:**
    -
- **Actions taken:**
    - 

In [ ]:
# Basic EDA

tb.eda_column_summary(df, 'Total years of experience')

###  'Years of experience in Germany'
- **Definition:**
    - 
- **Inconsistencies:**
    -
- **Actions taken:**
    - 

In [38]:
# Basic EDA

tb.eda_column_summary(df, 'Years of experience in Germany')

=== COLUMN: Years of experience in Germany ===

DTYPE:
float64

NON-NULL VALUES:
1,195

NULL VALUES:
58

BASIC STATS:
count    1195.000000
mean        3.754812
std         3.656636
min         0.000000
25%         1.000000
50%         3.000000
75%         5.000000
max        30.000000
Name: Years of experience in Germany, dtype: float64

TOP VALUES:
Years of experience in Germany
2.0    195
1.0    189
3.0    155
5.0    142
4.0    122
Name: count, dtype: int64

BOTTOM VALUES:
Years of experience in Germany
26.0    1
4.5     1
0.1     1
2.6     1
0.2     1
Name: count, dtype: int64


###  'Seniority level'
- **Definition:**
    - 
- **Inconsistencies:**
    -
- **Actions taken:**
    - 

In [39]:
# Basic EDA

tb.eda_column_summary(df, 'Seniority level')

=== COLUMN: Seniority level ===

DTYPE:
string

NON-NULL VALUES:
1,241

NULL VALUES:
12

BASIC STATS:
count       1241
unique        21
top       senior
freq         565
Name: Seniority level, dtype: object

TOP VALUES:
Seniority level
senior    565
middle    366
lead      166
junior     79
head       44
Name: count, dtype: Int64

BOTTOM VALUES:
Seniority level
no idea, there are no ranges in the firm    1
c-level                                     1
working student                             1
entry level                                 1
self employed                               1
Name: count, dtype: Int64


###  'Your main technology / programming language'
- **Definition:**
    - 
- **Inconsistencies:**
    -
- **Actions taken:**
    - 

In [40]:
# Basic EDA

tb.eda_column_summary(df, 'Your main technology / programming language')

=== COLUMN: Your main technology / programming language ===

DTYPE:
string

NON-NULL VALUES:
1,125

NULL VALUES:
128

BASIC STATS:
count       1125
unique       206
top       python
freq         212
Name: Your main technology / programming language, dtype: object

TOP VALUES:
Your main technology / programming language
python        212
java          196
javascript     80
php            71
c++            41
Name: count, dtype: Int64

BOTTOM VALUES:
Your main technology / programming language
.net, c++, spss, embeddded    1
.net, angular                 1
atlassian jira                1
qlik                          1
consumer analysis             1
Name: count, dtype: Int64


###  'Other technologies/programming languages you use often'
- **Inconsistencies:**
    -
- **Actions taken:**
    - 

In [41]:
# Basic EDA

tb.eda_column_summary(df, 'Other technologies/programming languages you use often')

=== COLUMN: Other technologies/programming languages you use often ===

DTYPE:
string

NON-NULL VALUES:
1,096

NULL VALUES:
157

BASIC STATS:
count                        1096
unique                        561
top       javascript / typescript
freq                           44
Name: Other technologies/programming languages you use often, dtype: object

TOP VALUES:
Other technologies/programming languages you use often
javascript / typescript    44
python                     37
sql                        31
aws, docker                16
kotlin                     15
Name: count, dtype: Int64

BOTTOM VALUES:
Other technologies/programming languages you use often
python, kotlin, ruby, aws, docker                               1
c/c++, .net, go                                                 1
c/c++, .net                                                     1
python, c/c++, rust, aws, kubernetes, docker                    1
python, c/c++, javascript / typescript, sql, go, aws, docker    1
N

###  'Yearly brutto salary (without bonus and stocks) in EUR'
- **Definition:**
    - 
- **Inconsistencies:**
    -
- **Actions taken:**
    - 

In [42]:
# Basic EDA

tb.eda_column_summary(df, 'Yearly brutto salary (without bonus and stocks) in EUR')

=== COLUMN: Yearly brutto salary (without bonus and stocks) in EUR ===

DTYPE:
float64

NON-NULL VALUES:
1,253

NULL VALUES:
0

BASIC STATS:
count    1.253000e+03
mean     8.027904e+07
std      2.825061e+09
min      1.000100e+04
25%      5.880000e+04
50%      7.000000e+04
75%      8.000000e+04
max      1.000000e+11
Name: Yearly brutto salary (without bonus and stocks) in EUR, dtype: float64

TOP VALUES:
Yearly brutto salary (without bonus and stocks) in EUR
60000.0    85
70000.0    83
65000.0    77
75000.0    73
80000.0    66
Name: count, dtype: int64

BOTTOM VALUES:
Yearly brutto salary (without bonus and stocks) in EUR
68250.0     1
84700.0     1
50500.0     1
108500.0    1
38350.0     1
Name: count, dtype: int64


###  'Yearly bonus + stocks in EUR'
- **Definition:**
    - 
- **Inconsistencies:**
    -
- **Actions taken:**
    - 

In [43]:
# Basic EDA

tb.eda_column_summary(df, 'Yearly bonus + stocks in EUR')

=== COLUMN: Yearly bonus + stocks in EUR ===

DTYPE:
float64

NON-NULL VALUES:
823

NULL VALUES:
430

BASIC STATS:
count    8.230000e+02
mean     6.096870e+06
std      1.742883e+08
min      0.000000e+00
25%      0.000000e+00
50%      5.000000e+03
75%      2.000000e+04
max      5.000000e+09
Name: Yearly bonus + stocks in EUR, dtype: float64

TOP VALUES:
Yearly bonus + stocks in EUR
0.0        227
5000.0      56
10000.0     45
2000.0      36
6000.0      26
Name: count, dtype: int64

BOTTOM VALUES:
Yearly bonus + stocks in EUR
110000.0    1
700.0       1
16320.0     1
105000.0    1
40950.0     1
Name: count, dtype: int64


###  'Number of vacation days'
- **Definition:**
    - 
- **Inconsistencies:**
    -
- **Actions taken:**
    - 

In [44]:
# Basic EDA

tb.eda_column_summary(df, 'Number of vacation days')

=== COLUMN: Number of vacation days ===

DTYPE:
float64

NON-NULL VALUES:
1,172

NULL VALUES:
81

BASIC STATS:
count    1172.000000
mean       28.348549
std        10.759972
min         0.000000
25%        27.000000
50%        28.000000
75%        30.000000
max       365.000000
Name: Number of vacation days, dtype: float64

TOP VALUES:
Number of vacation days
30.0    488
28.0    233
27.0    102
25.0     91
26.0     71
Name: count, dtype: int64

BOTTOM VALUES:
Number of vacation days
5.0     1
37.0    1
39.0    1
34.0    1
10.0    1
Name: count, dtype: int64


###  'Employment status'
- **Definition:**
    - 
- **Inconsistencies:**
    -
- **Actions taken:**
    - 

In [45]:
# Basic EDA

tb.eda_column_summary(df, 'Employment status')

=== COLUMN: Employment status ===

DTYPE:
string

NON-NULL VALUES:
1,236

NULL VALUES:
17

BASIC STATS:
count                   1236
unique                    10
top       full-time employee
freq                    1190
Name: Employment status, dtype: object

TOP VALUES:
Employment status
full-time employee            1190
self-employed (freelancer)      28
part-time employee               8
founder                          3
working student                  2
Name: count, dtype: Int64

BOTTOM VALUES:
Employment status
company director                                                                   1
full-time position, part-time position, & self-employed (freelancing, tutoring)    1
intern                                                                             1
full-time, but 32 hours per week (it was my request, i'm a student)                1
werkstudent                                                                        1
Name: count, dtype: Int64


###  'Сontract duration'
- **Definition:**
    - 
- **Inconsistencies:**
    -
- **Actions taken:**
    - 

In [46]:
# Basic EDA

tb.eda_column_summary(df, 'Сontract duration')

KeyError: 'Сontract duration'

###  'Main language at work'
- **Definition:**
    - 
- **Inconsistencies:**
    -
- **Actions taken:**
    - 

In [47]:
# Basic EDA

tb.eda_column_summary(df, 'Main language at work')

=== COLUMN: Main language at work ===

DTYPE:
string

NON-NULL VALUES:
1,237

NULL VALUES:
16

BASIC STATS:
count        1237
unique         14
top       english
freq         1020
Name: Main language at work, dtype: object

TOP VALUES:
Main language at work
english    1020
german      186
russian      12
italian       3
spanish       3
Name: count, dtype: Int64

BOTTOM VALUES:
Main language at work
50/50               1
deuglisch           1
french              1
both                1
russian, english    1
Name: count, dtype: Int64


###  'Company size'
- **Definition:**
    - 
- **Inconsistencies:**
    -
- **Actions taken:**
    - 

In [48]:
# Basic EDA

tb.eda_column_summary(df, 'Company size')

=== COLUMN: Company size ===

DTYPE:
string

NON-NULL VALUES:
1,235

NULL VALUES:
18

BASIC STATS:
count      1235
unique        5
top       1000+
freq        448
Name: Company size, dtype: object

TOP VALUES:
Company size
1000+       448
101-1000    405
11-50       174
51-100      147
up to 10     61
Name: count, dtype: Int64

BOTTOM VALUES:
Company size
1000+       448
101-1000    405
11-50       174
51-100      147
up to 10     61
Name: count, dtype: Int64


###  'Company type'
- **Definition:**
    - 
- **Inconsistencies:**
    -
- **Actions taken:**
    - 

In [84]:
column = 'Company type'

In [86]:
# Basic EDA

tb.eda_column_summary(df, column)

=== COLUMN: Company type ===

DTYPE:
string

NON-NULL VALUES:
1,228

NULL VALUES:
25

BASIC STATS:
count        1228
unique         57
top       product
freq          760
Name: Company type, dtype: object

TOP VALUES:
Company type
product                760
startup                252
consulting / agency    142
bank                     6
e-commerce               5
Name: count, dtype: Int64

BOTTOM VALUES:
Company type
industry        1
faang           1
old industry    1
outsourse       1
freelance       1
Name: count, dtype: Int64


In [87]:
df[column].value_counts()

Company type
product                                                                     760
startup                                                                     252
consulting / agency                                                         142
bank                                                                          6
e-commerce                                                                    5
media                                                                         3
corporation                                                                   3
automotive                                                                    2
university                                                                    2
fintech                                                                       2
ecommerce                                                                     2
finance                                                                       2
utilities                  

In [83]:
tb.plot(df, columns=column)

# Answers and Explainations
(Expand/modify as needed)

### Here's the Data Cleaning Steps I Used

### Here's my Tuning/Feature Selection Steps

### Here's my Model's Performance

### Here's my Final Conclusion on What Worked Best